In [6]:
########################## Proceso ETL - Huawei 3G ##########################
# MA. 20250923.

# Librerías
import pandas as pd
import numpy as np
import re
import glob
import os
import datetime
from datetime import datetime
from datetime import date
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl import load_workbook


## Variables compartidas

# Ubicación archivos
ruta_carpeta = "C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Huawei/Huawei-Feb"

ruta_ept = "C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Huawei/Huawei-Feb"

ruta_destino = "C:/Users/SCaracoza/Documents/AT&T/LST Cell Ran/Huawei/Huawei-Feb"

# Fecha para el nombre de los archivos a crear
fecha_ejecucion: str = datetime.now().strftime("%Y%m")

In [18]:
# MA. 20250926
### LST UCELL. Lectura y unificación de archivos. ###

# Prefijo de archivos
archivo_prefijo = "MML_Task_Result_LST UCELL_c"
archivos_txt = glob.glob(os.path.join(ruta_carpeta, f"{archivo_prefijo}*.txt"))

# Regex
pat_section = re.compile(r"(MML Command-----LST UCELL.*?)(?=MML Command-----LST UCELL|$)", re.S)
pat_ne = re.compile(r"NE\s*:\s*(\S+)")
pat_plus = re.compile(r"^\+\+\+\s+(\S+)", re.M)

# Columnas estándar
columnas_lst = [
    "Origen", "RNC", "Logical RNC ID", "Cell ID", "Cell Name", "Max Transmit Power of Cell",
    "Band Indicator", "Cn Operator Group Index", "UL Frequency Ind", "Uplink UARFCN",
    "Downlink UARFCN", "Time Offset", "Num of Continuous in Sync Ind",
    "Num of Continuous Out of Sync Ind", "Radio Link Failure Timer Length",
    "DL Power Control Mode 1", "DL Primary Scrambling Code", "TX Diversity Indication",
    "Service Priority Group Identity", "NodeB Name", "Local Cell ID",
    "Location Area Code", "Service Area Code", "RAC Configuration Indication",
    "Routing Area Code", "STTD Support Indicator", "CP1 Support Indicator",
    "Closed Loop Time Adjust Mode", "DPCH Tx Diversity Mode for Other User",
    "FDPCH Tx Diversity Mode for Other User", "DPCH Tx Diversity Mode for MIMO User",
    "FDPCH Tx Diversity Mode for MIMO User", "Tx Diversity Mode for DC-HSDPA User",
    "Cell Oriented Cell Individual Offset", "Cell VP Limit Indicator", "DSS Cell Flag",
    "Maximum TX Power in Small DSS Coverage", "Common Channel Bandwidth Operator Index",
    "Hierarchy ID of Terminal Type", "Heterogeneous Cell Flag", "HostType",
    "Validation indication", "Cell administrative state", "Cell MIMO state",
    "IPDL flag", "Cell CBS state", "Cell ERACH state",
    "Subrack No.", "Subrack name", "Slot No.", "Subsystem No.", "Cell MBMS state",
    "SSN"
]

dfs = []

for archivo in archivos_txt:
    with open(archivo, "r", encoding="utf-8", errors="ignore") as f:
        contenido = f.read()

    # Divide por secciones de MML Command
    secciones = pat_section.findall(contenido)

    for seccion in secciones:
        # Detecta NE
        m_ne = pat_ne.search(seccion)
        ne_actual = m_ne.group(1) if m_ne else ""

        # Si NE no existe, intenta con +++
        if not ne_actual:
            m_plus = pat_plus.search(seccion)
            if m_plus:
                ne_actual = m_plus.group(1)

        # Divide en bloques de List Cell Basic Information
        bloques = re.split(r"List Cell Basic Information\s*-+\s*", seccion)[1:]

        for bloque in bloques:
            lineas = bloque.strip().splitlines()
            if len(lineas) < 2:
                continue

            encabezado = re.split(r"\s{2,}", lineas[0].strip())
            datos = []

            for linea in lineas[1:]:
                if linea.strip().startswith(("---", "+++", "Number of results", "To be continued")):
                    break
                if not linea.strip():
                    continue
                fila = re.split(r"\s{2,}", linea.strip())
                if len(fila) < len(encabezado):
                    fila += [""] * (len(encabezado) - len(fila))
                elif len(fila) > len(encabezado):
                    fila = fila[:len(encabezado)]
                datos.append(fila)

            if datos:
                df_tmp = pd.DataFrame(datos, columns=encabezado)
                df_tmp["Origen"] = os.path.basename(archivo)
                df_tmp["RNC"] = ne_actual

                # Normaliza columnas
                for col in columnas_lst:
                    if col not in df_tmp.columns:
                        df_tmp[col] = ""

                df_tmp = df_tmp[columnas_lst]
                dfs.append(df_tmp)

# Unión
df_final = pd.concat(dfs, ignore_index=True, sort=False) if dfs else pd.DataFrame(columns=columnas_lst)

# --- Limpieza y transformaciones ---
df_final.columns = df_final.columns.str.strip().str.replace(r"\s+", " ", regex=True)
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# Reemplaza <NULL> en Uplink UARFCN
if "Uplink UARFCN" in df_final.columns:
    df_final["Uplink UARFCN"] = df_final["Uplink UARFCN"].replace("<NULL>", "0")

# Asegura columnas de SSN
for col in ["Subrack No.", "Slot No.", "Subsystem No."]:
    if col not in df_final.columns:
        df_final[col] = ""

# Genera SSN
df_final["SSN"] = (
    df_final[["Subrack No.", "Slot No.", "Subsystem No."]]
    .astype(str).fillna("")
    .agg("-".join, axis=1)
)
df_final["SSN"] = df_final["SSN"].apply(lambda x: "--" if x.replace("-", "").strip() == "" else x)

# Filtrado
if "Logical RNC ID" in df_final.columns:
    df_final = df_final[~df_final["Logical RNC ID"].str.contains(r"\+\+\+", na=False)]
if "Cell Name" in df_final.columns:
    df_final = df_final[df_final["Cell Name"].astype(str).str.strip() != ""]

# Convierte columnas numéricas cuando aplica
df_LST_UCELL_inicial = df_final.apply(
    lambda col: pd.to_numeric(col, errors="coerce")
    if not pd.to_numeric(col, errors="coerce").isna().any()
    else col
)

# Archivo unificado en Excel (validación)
#salida_excel = os.path.join(ruta_destino, f"LST_UCELL_{fecha_ejecucion}.xlsx")
#df_LST_UCELL_inicial[columnas_lst].to_excel(salida_excel, index=False, engine="openpyxl")

KeyboardInterrupt: 

In [11]:
# MA. 20250929
### DSP UCELL. Lectura y unificación de archivos. ###

# Prefijo de archivos
archivo_prefijo_dsp = "MML_Task_Result_DSP UCELL_"
archivos_dsp = glob.glob(os.path.join(ruta_carpeta, f"{archivo_prefijo_dsp}*.txt"))

# Columnas esperadas
columnas_dsp = [
    "Cell ID", "Cell Name", "Operation state", "Administrative state",
    "DSS state", "State explanation", "Subrack No.", "Slot No.", "Subsystem No."
]

dfs_dsp = []

if archivos_dsp:
    for archivo in archivos_dsp:
        with open(archivo, "r", encoding="utf-8", errors="ignore") as f:
            contenido = f.read()

        # Divide en bloques a partir de "Cell state information"
        bloques = re.split(r"Cell state information\s*-+\s*", contenido)[1:]

        for bloque in bloques:
            lineas = bloque.strip().splitlines()
            if len(lineas) < 2:
                continue

            encabezado = re.split(r"\s{2,}", lineas[0].strip())
            datos = []

            for linea in lineas[1:]:
                if linea.strip().startswith(("---", "+++", "To be continued", "Number of results")):
                    break
                if not linea.strip():
                    continue

                fila = re.split(r"\s{2,}", linea.strip())
                # Normaliza longitud
                if len(fila) < len(encabezado):
                    fila += [""] * (len(encabezado) - len(fila))
                elif len(fila) > len(encabezado):
                    fila = fila[:len(encabezado)]
                datos.append(fila)

            if datos:
                df_tmp = pd.DataFrame(datos, columns=encabezado)
                df_tmp["Origen"] = os.path.basename(archivo)
                dfs_dsp.append(df_tmp)

if dfs_dsp:
    df_DSP_UCELL_ini = pd.concat(dfs_dsp, ignore_index=True, sort=True)

    # Renombra "Cell name" -> "Cell Name"
    if "Cell name" in df_DSP_UCELL_ini.columns:
        df_DSP_UCELL_ini.rename(columns={"Cell name": "Cell Name"}, inplace=True)

    # Asegura columnas esperadas
    for col in columnas_dsp:
        if col not in df_DSP_UCELL_ini.columns:
            df_DSP_UCELL_ini[col] = ""

    # Reordena
    df_DSP_UCELL_inicial = df_DSP_UCELL_ini[columnas_dsp + ["Origen"]]

    # Elimina registros donde "Cell Name" esté vacío o NaN
    if "Cell Name" in df_DSP_UCELL_inicial.columns:
        df_DSP_UCELL_inicial = df_DSP_UCELL_inicial[df_DSP_UCELL_inicial["Cell Name"].astype(str).str.strip() != ""]

    # Convierte columnas numéricas cuando aplica
    df_DSP_UCELL_inicial = df_DSP_UCELL_inicial.apply(
        lambda col: pd.to_numeric(col, errors="coerce")
        if not pd.to_numeric(col, errors="coerce").isna().any()
        else col
    )
else:
    df_DSP_UCELL_inicial = pd.DataFrame(columns=columnas_dsp + ["Origen"])


# Archivo unificado en Excel (validación)
#ruta_salida = os.path.join(ruta_destino, f"DSP_UCELL_{fecha_ejecucion}.xlsx")
#df_DSP_UCELL_inicial.to_excel(ruta_salida, index=False, engine="openpyxl")

In [12]:
# MA. 20250923.
### LST UCELLURA. Lectura y unificación de archivos. ###

# Prefijo de archivos
archivo_prefijo_ura = "MML_Task_Result_LST UCELLURA_c"
archivos_ura = glob.glob(os.path.join(ruta_carpeta, f"{archivo_prefijo_ura}*.txt"))

# Columnas esperadas
columnas_ura = ["Origen", "NE", "Logical RNC ID", "Cell ID", "Cell Name", "URA ID"]

dfs_ura = []

if archivos_ura:
    for archivo in archivos_ura:
        with open(archivo, "r", encoding="utf-8", errors="ignore") as f:
            contenido = f.read()

        # Detecta NE (si viene vacío, lo toma de +++)
        ne_actual = None
        m_ne = re.search(r"^NE\s*:\s*(\S+)?", contenido, re.MULTILINE)
        if m_ne and m_ne.group(1):
            ne_actual = m_ne.group(1)
        else:
            m_plus = re.search(r"^\+\+\+\s+(\S+)", contenido, re.MULTILINE)
            if m_plus:
                ne_actual = m_plus.group(1)

        # Divide por bloques de tabla "List Cell URA"
        bloques = re.split(r"List Cell URA\s*-+\s*", contenido)[1:]

        for bloque in bloques:
            lineas = bloque.strip().splitlines()
            if len(lineas) < 2:
                continue

            # Encabezado y datos
            encabezado = re.split(r"\s{2,}", lineas[0].strip())
            datos = []
            for linea in lineas[1:]:
                if linea.strip().startswith(("---", "+++", "Number of results", "To be continued")):
                    break
                if not linea.strip():
                    continue
                fila = re.split(r"\s{2,}", linea.strip())
                if len(fila) < len(encabezado):
                    fila += [""] * (len(encabezado) - len(fila))
                elif len(fila) > len(encabezado):
                    fila = fila[:len(encabezado)]
                datos.append(fila)

            if datos:
                df_tmp = pd.DataFrame(datos, columns=encabezado)
                df_tmp["Origen"] = os.path.basename(archivo)
                df_tmp["NE"] = ne_actual
                dfs_ura.append(df_tmp)

# Une resultados
if dfs_ura:
    df_LST_UCELLURA_ini = pd.concat(dfs_ura, ignore_index=True, sort=True)

    # Reordena y garantiza columnas esperadas
    for col in columnas_ura:
        if col not in df_LST_UCELLURA_ini.columns:
            df_LST_UCELLURA_ini[col] = ""

    df_LST_UCELLURA_inicial = df_LST_UCELLURA_ini[columnas_ura]


# Elimina registros donde "Cell Name" esté vacío o NaN
    if "Cell Name" in df_LST_UCELLURA_inicial.columns:
        df_LST_UCELLURA_inicial = df_LST_UCELLURA_inicial[
        df_LST_UCELLURA_inicial["Cell Name"].astype(str).str.strip() != ""
    ]
else:
    df_LST_UCELLURA_inicial = pd.DataFrame(columns=columnas_ura)

# Convierte columnas numéricas si aplica
df_LST_UCELLURA_inicial = df_LST_UCELLURA_inicial.apply(
    lambda col: pd.to_numeric(col, errors="coerce")
    if not pd.to_numeric(col, errors="coerce").isna().any()
    else col
)

# Archivo unificado en CSV, para fines de validación.
#ruta_salida = os.path.join(ruta_destino, f"LST_UCELLURA_{fecha_ejecucion}.xlsx")
#df_LST_UCELLURA_inicial.to_excel(ruta_salida, index=False, engine="openpyxl")

In [13]:
# MA. 20250924.
### Información archivo mes anterior ###

# Sufijo mes anterior
today = date.today()
prev_year  = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm = f"{prev_year}{prev_month:02d}"
fecha_hoy = today.strftime("%d/%m/%Y")

# Busca archivo
ruta: str = ruta_destino  # --> MA. Por Definir.
archivo = f"All_Huawei_3G_{yyyymm}.xlsx"
path = os.path.join(ruta, archivo)

# Columnas necesarias
columnas_anteriores = ["ANT Cell Id", "ANT Cell Name", "RNC", "LAT", "LON", "AT&T_Site_Name", "En el gestor"]

if os.path.exists(path):
    # Fecha del día y de creación de archivo anterior
    wb = load_workbook(path, read_only=True)
    props = wb.properties
    fecha_creacion = props.created.strftime("%d/%m/%Y")

    # Extrae columnas necesarias
    df_All_Huawei_3G_Anterior = pd.read_excel(
        path,
        usecols=["Cell Id", "Cell Name", "LAT", "LON", "AT&T_Site_Name", "En el gestor"]
    )

    # Renombra columnas llave
    df_All_Huawei_3G_Anterior = df_All_Huawei_3G_Anterior.rename(
        columns={"Cell Id": "ANT Cell Id", "Cell Name": "ANT Cell Name"}
    )

    ## Formato columna "En el gestor"
    df_All_Huawei_3G_Anterior["En el gestor"] = df_All_Huawei_3G_Anterior["En el gestor"].apply(
    lambda x: pd.to_datetime(x, dayfirst=True, errors="coerce")
              .strftime("%d/%m/%Y") if not pd.isna(pd.to_datetime(x, dayfirst=True, errors="coerce")) else x
        )

else:
    # No existe archivo, crea DataFrame vacío con columnas necesarias
    df_All_Huawei_3G_Anterior = pd.DataFrame(columns=columnas_anteriores)

In [14]:
# MA. 20250926.
### Información del EPT ###

# Prefijo del archivo
prefijo_ept = "EPT_ATT_UMTS_LTE_"

# Busca archivo que empiece con el prefijo
archivo = glob.glob(os.path.join(ruta_ept, f"{prefijo_ept}*.xlsx"))

# Inicializa por si no hay archivo
df_EPT_ini = pd.DataFrame()
df_EPT_unificado = pd.DataFrame(columns=["Cell Name", "LAT", "LON", "AT&T_Site_Name"])
df_EPT_nodes = pd.DataFrame(columns=["node_key", "LAT", "LON", "AT&T_Site_Name"])

if archivo:
    archivo_encontrado = archivo[0]
    nombre_archivo = os.path.basename(archivo_encontrado)

    # Lista de hojas a leer
    hojas = [
        "EPT_3G_LTE_OUTDOOR",
        "PLAN_OUTDOOR",
        "EPT_3G_LTE_INDOOR",
        "PLAN_INDOOR",
        "Eventos_Especiales"
    ]

    # Lee todas las hojas y agrega el nombre de la hoja en columna
    dfs = [
        pd.read_excel(archivo_encontrado, sheet_name=hoja, engine="openpyxl")
        .assign(Hoja=hoja, Origen=nombre_archivo)
        for hoja in hojas
    ]

    # Concatena todo en un solo DataFrame
    df_EPT_inicial = pd.concat(dfs, ignore_index=True)

    # Elimina duplicados
    df_EPT_inicial = df_EPT_inicial.drop_duplicates().reset_index(drop=True)

    # Renombramiento columna(s)
    nuevos_nombres = {"CellName": "Cell Name", "Latitud": "LAT", "Longitud": "LON"}
    df_EPT_inicial.rename(columns=nuevos_nombres, inplace=True)

    # Convierte columnas totalmente numéricas (cuando aplica)
    df_EPT_ini = df_EPT_inicial.apply(
        lambda col: pd.to_numeric(col, errors="coerce")
        if not pd.to_numeric(col, errors="coerce").isna().any()
        else col
    )

    # -------------------------------
    # 1) EPT por Cell Name (como ya tenías)
    # -------------------------------
    df_EPT_unificado = df_EPT_ini[["Cell Name", "LAT", "LON", "AT&T_Site_Name"]]

    # -------------------------------
    # 2) EPT por NodeB Name (preparado aquí)
    #    Genera una tabla "larga" con AT&T_Node_Name y Node_B_U2000
    # -------------------------------
    cols_nodo_ept = [c for c in ["AT&T_Node_Name", "Node_B_U2000"] if c in df_EPT_ini.columns]
    cols_site = [c for c in ["LAT", "LON", "AT&T_Site_Name"] if c in df_EPT_ini.columns]

    if cols_nodo_ept and set(["LAT", "LON", "AT&T_Site_Name"]).issubset(df_EPT_ini.columns):

        df_ept_nodes_raw = (
            df_EPT_ini[cols_site + cols_nodo_ept]
            .melt(id_vars=cols_site, value_vars=cols_nodo_ept, value_name="node_name")
            .drop(columns=["variable"])
            .dropna(subset=["node_name"])
            .drop_duplicates()
            .reset_index(drop=True)
        )

        # Normalizador consistente
        def _norm(s: str) -> str:
            s = str(s)
            return (s.upper()
                      .replace(" ", "")
                      .replace("_", "")
                      .replace("-", "")
                      .replace(".", "")
                      .strip())

        df_ept_nodes_raw["node_key"] = df_ept_nodes_raw["node_name"].map(_norm)

        # Mantén una fila por node_key (si hay duplicados, toma la primera con datos)
        df_EPT_nodes = (
            df_ept_nodes_raw
            .drop_duplicates(subset=["node_key"], keep="first")
            .loc[:, ["node_key", "LAT", "LON", "AT&T_Site_Name"]]
            .reset_index(drop=True)
        )


In [17]:
# MA. 20250923.
### Creación archivo final ###

## Extraemos solo las columnas requeridas de cada DataFrame.

## LST_UCELL
df_LST_UCELL = df_LST_UCELL_inicial[[
    "RNC","Logical RNC ID","Cell ID","Cell Name","Max Transmit Power of Cell","Band Indicator",
    "Cn Operator Group Index","UL Frequency Ind","Uplink UARFCN","Downlink UARFCN","Time Offset",
    "Num of Continuous in Sync Ind","Num of Continuous Out of Sync Ind","Radio Link Failure Timer Length",
    "DL Power Control Mode 1","DL Primary Scrambling Code","TX Diversity Indication",
    "Service Priority Group Identity","NodeB Name","Local Cell ID","Location Area Code","Service Area Code",
    "RAC Configuration Indication","Routing Area Code","STTD Support Indicator","CP1 Support Indicator",
    "Closed Loop Time Adjust Mode","DPCH Tx Diversity Mode for Other User","FDPCH Tx Diversity Mode for Other User",
    "DPCH Tx Diversity Mode for MIMO User","FDPCH Tx Diversity Mode for MIMO User","Tx Diversity Mode for DC-HSDPA User",
    "Cell Oriented Cell Individual Offset","Cell VP Limit Indicator","DSS Cell Flag",
    "Maximum TX Power in Small DSS Coverage","Common Channel Bandwidth Operator Index",
    "Hierarchy ID of Terminal Type","Heterogeneous Cell Flag","HostType","Validation indication",
    "Cell administrative state","Cell MIMO state","IPDL flag","Cell CBS state","Cell ERACH state",
    "Subrack No.","Subrack name","Slot No.","Subsystem No.","Cell MBMS state","SSN"
]]
df_LST_UCELL = df_LST_UCELL.rename(columns={"Logical RNC ID": "RNC ID"})
# >>> NUEVO: columna Gestor a partir de 'Origen' del LST <<<
# 'Origen' existe en df_LST_UCELL_inicial porque lo agregaste al leer cada archivo.
_gestor = df_LST_UCELL_inicial["Origen"].str.extract(r"LST UCELL_c(\d+)", expand=True).iloc[:, 0]
df_LST_UCELL["Gestor"] = np.where(_gestor.notna() & (_gestor != ""), "MAE-" + _gestor.astype(str), "")


## DSP_UCELL
df_DSP_UCELL = df_DSP_UCELL_inicial[[
    "Cell ID","Cell Name","Operation state","Administrative state","State explanation"
]]

## LST_UCELLURA
df_LST_UCELLURA = df_LST_UCELLURA_inicial[["Cell ID","Cell Name","URA ID"]]
df_LST_UCELLURA = df_LST_UCELLURA.rename(columns={"URA ID": "URA"})

## Joins base
df_Huawei_3G_inicial = df_LST_UCELL.merge(df_DSP_UCELL, on=["Cell ID","Cell Name"], how="left")
df_Huawei_3G_inicial = df_Huawei_3G_inicial.merge(df_LST_UCELLURA, on=["Cell ID","Cell Name"], how="left")

## Archivo anterior
df_Huawei_3G_inicial = pd.merge(
    df_Huawei_3G_inicial, df_All_Huawei_3G_Anterior,
    left_on=["Cell ID","Cell Name"], right_on=["ANT Cell Id","ANT Cell Name"],
    how="left"
)

## EPT por Cell Name
df_EPT_unificado = (
    df_EPT_unificado
      .replace("", np.nan)
      .sort_values(by=["Cell Name"])
      .drop_duplicates(subset=["Cell Name"], keep="first")
)


# Fallback por Cell Name
fallback_map = {
    "LAT": "LAT_EPT",
    "LON": "LON_EPT",
    "AT&T_Site_Name": "AT&T_Site_Name_EPT"
}

for col, col_ept in fallback_map.items():
    if col in df_Huawei_3G_inicial.columns and col_ept in df_Huawei_3G_inicial.columns:
        # EPT = fuente primaria, AE (col) = respaldo
        ept_vals = df_Huawei_3G_inicial[col_ept].replace("", np.nan)
        ae_vals  = df_Huawei_3G_inicial[col].replace("", np.nan)
        df_Huawei_3G_inicial[col] = ept_vals.combine_first(ae_vals)

# (Opcional) Limpia columnas *_EPT
df_Huawei_3G_inicial.drop(columns=["LAT_EPT","LON_EPT","AT&T_Site_Name_EPT"],
                          inplace=True, errors="ignore")

## Fallback adicional por NodeB Name usando df_EPT_nodes (preparado en el bloque EPT)
if 'df_EPT_nodes' in globals() and not df_EPT_nodes.empty:
    def _norm(s: str) -> str:
        s = str(s)
        return (s.upper().replace(" ","").replace("_","").replace("-","").replace(".","").strip())

    # Solo filas que aún falten LAT/LON/Site
    mask_missing = (
        df_Huawei_3G_inicial[["LAT","LON","AT&T_Site_Name"]]
        .replace("", np.nan).isna().any(axis=1)
    )
    if mask_missing.any():
        df_Huawei_3G_inicial.loc[mask_missing, "node_key"] = (
            df_Huawei_3G_inicial.loc[mask_missing, "NodeB Name"].astype(str).map(_norm)
        )

        # Trae LAT/LON/Site por node_key
        df_tmp = df_Huawei_3G_inicial.loc[mask_missing, ["node_key"]].merge(
            df_EPT_nodes.rename(columns={
                "LAT":"LAT_ept_nb",
                "LON":"LON_ept_nb",
                "AT&T_Site_Name":"AT&T_Site_Name_ept_nb"
            }),
            on="node_key", how="left"
        )

        # Escribe columnas temporales en el mismo slice
        df_Huawei_3G_inicial.loc[mask_missing, ["LAT_ept_nb","LON_ept_nb","AT&T_Site_Name_ept_nb"]] = \
            df_tmp[["LAT_ept_nb","LON_ept_nb","AT&T_Site_Name_ept_nb"]].values

        # Rellena solo lo que sigue vacío
        for col, nb_col in [("LAT","LAT_ept_nb"), ("LON","LON_ept_nb"), ("AT&T_Site_Name","AT&T_Site_Name_ept_nb")]:
            if col in df_Huawei_3G_inicial.columns and nb_col in df_Huawei_3G_inicial.columns:
                df_Huawei_3G_inicial[col] = (
                    df_Huawei_3G_inicial[col].replace("", np.nan).fillna(df_Huawei_3G_inicial[nb_col])
                )

        # Limpieza
        df_Huawei_3G_inicial.drop(columns=["node_key","LAT_ept_nb","LON_ept_nb","AT&T_Site_Name_ept_nb"],
                                  inplace=True, errors="ignore")

## "En el gestor"
df_Huawei_3G_inicial["En el gestor"] = np.where(
    df_Huawei_3G_inicial["En el gestor"].isna() | (df_Huawei_3G_inicial["En el gestor"] == ""),
    fecha_hoy, df_Huawei_3G_inicial["En el gestor"]
)

## Orden
df_Huawei_3G_ordenado = df_Huawei_3G_inicial.sort_values(by=["RNC","NodeB Name","Local Cell ID"])

## Consolidado
df_Huawei_3G_ordenado["Consolidado"] = np.where(
    df_Huawei_3G_ordenado["NodeB Name"].str.len() == 10, "Si", "No"
)

## NodeB Unique
_name = df_Huawei_3G_ordenado["NodeB Name"].astype(str).fillna("").str.strip()
is_new = _name.ne(_name.shift())
df_Huawei_3G_ordenado["NodeB Unique"] = np.where(is_new & _name.ne(""), df_Huawei_3G_ordenado["NodeB Name"], "")

## Renombres finales
df_Huawei_3G_ordenado = df_Huawei_3G_ordenado.rename(columns={"Cell ID": "Cell Id"})

## Numeric-only casts
df_Huawei_3G_ordenado = df_Huawei_3G_ordenado.apply(
    lambda col: pd.to_numeric(col, errors="coerce")
    if not pd.to_numeric(col, errors="coerce").isna().any()
    else col
)

## Estructura final
df_Huawei_3G = df_Huawei_3G_ordenado[[
    "RNC","Cell Id","Cell Name","RNC ID","Max Transmit Power of Cell","Band Indicator",
    "Cn Operator Group Index","UL Frequency Ind","Uplink UARFCN","Downlink UARFCN","Time Offset",
    "Num of Continuous in Sync Ind","Num of Continuous Out of Sync Ind","Radio Link Failure Timer Length",
    "DL Power Control Mode 1","DL Primary Scrambling Code","TX Diversity Indication",
    "Service Priority Group Identity","NodeB Name","Local Cell ID","Location Area Code","Service Area Code",
    "RAC Configuration Indication","Routing Area Code","STTD Support Indicator","CP1 Support Indicator",
    "Closed Loop Time Adjust Mode","DPCH Tx Diversity Mode for Other User","FDPCH Tx Diversity Mode for Other User",
    "DPCH Tx Diversity Mode for MIMO User","FDPCH Tx Diversity Mode for MIMO User","Tx Diversity Mode for DC-HSDPA User",
    "Cell Oriented Cell Individual Offset","Cell VP Limit Indicator","DSS Cell Flag",
    "Maximum TX Power in Small DSS Coverage","Common Channel Bandwidth Operator Index",
    "Hierarchy ID of Terminal Type","Subrack No.","Subrack name","Slot No.","Subsystem No.",
    "Heterogeneous Cell Flag","HostType","Validation indication","Cell administrative state","Cell MBMS state",
    "Cell MIMO state","IPDL flag","Cell CBS state","Cell ERACH state","NodeB Unique","LAT","LON",
    "Operation state","Administrative state","State explanation","SSN","En el gestor","URA","Consolidado",
    "AT&T_Site_Name", "Gestor"
]]

## Archivo Final
ruta_salida = os.path.join(ruta_destino, f"All_Huawei_3G_{fecha_ejecucion}.xlsx")
df_Huawei_3G.to_excel(ruta_salida, index=False, engine="openpyxl")

# Rotar encabezados
wb = load_workbook(ruta_salida)
ws = wb.active
for col_num, column_title in enumerate(df_Huawei_3G.columns, 1):
    cell = ws[f"{get_column_letter(col_num)}1"]
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom")
wb.save(ruta_salida)
